## 4.1 Unsupervised Learning
https://www.geeksforgeeks.org/image-segmentation-using-k-means-clustering/

In [ ]:
import os
import csv

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
## Local Path ##
path = os.path.join('.', 'QC')

## Subfolders ##
train_test_path = os.path.join(path, 'split')

In [ ]:
with open('file.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(data)